In [1]:
# First version: 18th of March 2022
# Author: Evangelos Papoutsellis
# Copyright 2022 Science and Techonology Facilities Council

# This software was developed during the Math+ “Maths meets Image” hackathon 2022.

# Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless
# required by applicable law or agreed to in writing, software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License
# for the specific language governing permissions and limitations under the License.


from cil.utilities.noise import gaussian
from cil.utilities.display import show2D
from cil.optimisation.operators import FiniteDifferenceOperator, GradientOperator, BlockOperator
from cil.optimisation.functions import L1Norm, MixedL21Norm, L2NormSquared, BlockFunction
from cil.optimisation.algorithms import PDHG
from cil.utilities.jupyter import islicer
from cil.framework import ImageGeometry

import nibabel as nib
import os
import numpy as np


# Dynamic denoising with isotropic-spatial TV coupled anisotropic with temporal TV

$$ \underset{u}{\mathrm{argmin}} \big\{\frac{1}{2}\| u - g \|^{2} + \alpha\|\partial_{t} u\|_{1} + \beta\|\nabla u\|_{2} \big\}$$


$$ \underset{u}{\mathrm{argmin}} \mathcal{F}(Ku) + \mathcal{G}(u)$$


### Algorithm used :  Primal-Dual Hybrid algorithm

In [2]:
def plot_2d_image(idx,vol,title,clims=None,cmap="viridis"):
    """Customized version of subplot to plot 2D image"""
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar()
    plt.title(title)
    plt.axis("off")


def crop_and_fill(templ_im, vol):
    """Crop volumetric image data and replace image content in template image object"""
    # Get size of template image and crop
    idim_orig = templ_im.as_array().shape
    idim = (1,)*(3-len(idim_orig)) + idim_orig
    offset = (numpy.array(vol.shape) - numpy.array(idim)) // 2
    vol = vol[offset[0]:offset[0]+idim[0], offset[1]:offset[1]+idim[1], offset[2]:offset[2]+idim[2]]
    
    # Make a copy of the template to ensure we do not overwrite it
    templ_im_out = templ_im.copy()
    
    # Fill image content 
    templ_im_out.fill(numpy.reshape(vol, idim_orig))
    return(templ_im_out)

In [3]:

data_path = '/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/NOR/patient071/'
example_ni1 = os.path.join(data_path, 'image.nii.gz')
n1_img = nib.load(example_ni1).get_fdata() #get a numpy


In [4]:
dynamic_img = n1_img[:,:,5,0,:,0] # what are the last 3 slices???
print(" Shape is {}".format(dynamic_img.shape))

 Shape is (192, 256, 30)


### Make a cil object, Add gaussian noise

In [5]:

ig = ImageGeometry(voxel_num_x = dynamic_img.shape[1], 
                   voxel_num_y = dynamic_img.shape[0], 
                   channels = 30)
tmp_data = ig.allocate()
tmp_data.fill(np.moveaxis(dynamic_img, 2, 0)) # change order of axis
tmp_data_res = tmp_data/tmp_data.max() 
noisy_data = gaussian(tmp_data_res, seed=10, var = 0.001) # add noise gaussian

In [6]:
islicer(tmp_data.array, direction=0)

interactive(children=(IntSlider(value=15, continuous_update=False, description='X', max=29), FloatRangeSlider(…

IntSlider(value=15, continuous_update=False, description='X', max=29)

In [7]:
alpha = 0.2
beta = 0.08
Dt = FiniteDifferenceOperator(ig, direction=0) # time is first
Grad = GradientOperator(ig, correlation="Space")

print(" Range of Dt = {}".format(Dt.range.shape))
print(" Range of Grad = {}".format(Grad.range.shape))

f1 = alpha * L1Norm()
f2 = beta * MixedL21Norm()
K = BlockOperator(Dt, Grad)
F = BlockFunction(f1, f2)
G = L2NormSquared(b = noisy_data)

Initialised GradientOperator with numpy backend
 Range of Dt = (30, 192, 256)
 Range of Grad = (2, 1)


/opt/SIRF-SuperBuild/INSTALL/python/cil/optimisation/operators/GradientOperator.py:93: UserWarning: Warning: correlation='Space' on multi-channel dataset will use `numpy` backend
  warnings.warn("Warning: correlation='Space' on multi-channel dataset will use `numpy` backend")


In [8]:
pdhg = PDHG(f=F, g=G, operator=K, max_iteration=100, 
            update_objective_interval=10, gamma_g=0.3)
pdhg.run(verbose=2)

PDHG setting up


/opt/SIRF-SuperBuild/INSTALL/python/cil/optimisation/algorithms/PDHG.py:362: UserWarning: Primal Acceleration of PDHG: The function g is assumed to be strongly convex with positive parameter `gamma_g`. You need to be sure that gamma_g = 0.3 is the correct strongly convex constant for g. 
  warnings.warn("Primal Acceleration of PDHG: The function g is assumed to be strongly convex with positive parameter `gamma_g`. You need to be sure that gamma_g = {} is the correct strongly convex constant for g. ".format(self.gamma_g))
/opt/SIRF-SuperBuild/INSTALL/python/cil/framework/framework.py:2381: RuntimeWarning: invalid value encountered in true_divide
  pwop(self.as_array(), x2.as_array(), *args, **kwargs )


PDHG configured
     Iter   Max Iter     Time/Iter        Primal          Dual     Primal-Dual
                               [s]     Objective     Objective             Gap
        0        100         0.000    2.03755e+04  -0.00000e+00    2.03755e+04
       10        100         0.106    2.96154e+03   2.44174e+03    5.19798e+02
       20        100         0.094    2.74181e+03   2.53879e+03    2.03024e+02
       30        100         0.090    2.68414e+03   2.56942e+03    1.14727e+02
       40        100         0.089    2.65720e+03   2.58309e+03    7.41143e+01
       50        100         0.088    2.64268e+03   2.59040e+03    5.22833e+01
       60        100         0.087    2.63392e+03   2.59476e+03    3.91572e+01
       70        100         0.086    2.62781e+03   2.59758e+03    3.02253e+01
       80        100         0.085    2.62351e+03   2.59952e+03    2.39972e+01
       90        100         0.085    2.62038e+03   2.60088e+03    1.95044e+01
      100        100         0.084  

In [9]:
sol = pdhg.solution
islicer(sol, direction=0)

interactive(children=(IntSlider(value=15, continuous_update=False, description='channel', max=29), FloatRangeS…

IntSlider(value=15, continuous_update=False, description='channel', max=29)